In [0]:
schema_name = 'artemis'
container = 'artemis'
storage_account_name = 'ryanstorageadls'
storage_account_key = ''

In [0]:
mount_location = "/mnt/{0}/{1}".format(storage_account_name, container)
if mount_location not in [f.mountPoint for f in dbutils.fs.mounts()]:
    dbutils.fs.mount(
    source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
    mount_point = mount_location,
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
    )
else :
    print("already mounted")

In [0]:
dbutils.fs.ls("/mnt/{0}/{1}".format(storage_account_name, container))

In [0]:
df = (spark
      .read
      .option('header', 'true')
      .option('inferSchema', 'true')
      .csv(mount_location+'/*.csv')
      )

In [0]:
display(df)

Databricks data profile. Run in Databricks to view.

In [0]:
df.count() # 1,449,224

In [0]:
df.write.saveAsTable(f'{schema_name}.miner_data')

In [0]:
min_date = df.agg({"time": "min"}).collect()[0][0]
max_date = df.agg({"time": "max"}).collect()[0][0]
print(f"{min_date} | {max_date}")

In [0]:
train_df = df.filter(df.time <= '2025-04-09')
test_df = df.filter(df.time > '2025-04-09')

In [0]:
train_df.write.saveAsTable(f'{schema_name}.training_data')
test_df.write.saveAsTable(f'{schema_name}.testing_data')